$$\require{mhchem}$$
# Aerobic Degradation of a Generic Organic carbon

In this notebook, we model the reactions involved in the aerobic degradation of a generic organic carbon.


## 1. Chemical Setup

The model is simplified to two chemical reactions:

1. Energy metabolism:

$ \ce{CH_2O + O_2 -> CO_2 + H_2O} $
***
2. Anabolism:

$ \ce{5CH_2O + NH_4^+ -> H^+ + C_5H_7O_2 + 3H_2O} $

Thus, we can see the organic carbon is consumed by both the oxidation and biomass production. Therefore the stoichiometry of the system must be evaluated according to the biomass efficiency:

Consider the molar mass of biomass: (113 g/mol). If we divide the second expression by the molar mass we get the stoichimetry of the reaction per gram of biomass:

Example: stoichimetry of $\ce{CH_2O}$

In [1]:
5/113

0.04424778761061947

2.1. Updated Stoichiometry of reaction:

$0.0442\ce{CH2O} + 0.0088 \ce{NH4+ ->} 0.0088 \ce{H+} + 0.0265 \ce{H2O} $

Now we need to consider the yield of the reaction (the amount of biomass generated by the consumption of the organic matter).

the Yield (**Y**) for this reaction will be:

In [2]:
Y = 17 # [g/molC_org]

$ Y = \frac{g_{biomass}}{C_{mol} consumed} = 1 g_{biomass}/(0.04424 + f)$

Where $f$ is the stoichiometry ratio of the energy metabolism part:

For this example we get:

In [3]:
f = 1/Y - 0.04424

In [4]:
f

0.014583529411764704

3. And the final reaction of the system:

$ 0.0442+0.0146\ce{CH2O} + 0.0146\ce{O2} + 0.0088 \ce{NH4+ ->} 1gbio + 0.0088 \ce{H+} + 0.0146\ce{CO2} + (0.0265+0.0146)\ce{H2O} $

## 2. Mathematical Setup

The reactions involved here are kinetically controlled. We will use a Monod kinetic formulation for the representation of the reaction rates and to model